In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from gee_scripts.get_sources import get_explanatory_composite
from gee_scripts.parameters import explain_vars

import ee
ee.Initialize()

# 1. Get explanatory composite

## 1.1. Set parameters

In [62]:
max_days_offset = 30
scale = 10
target_date = "2020-01-01"
output_image_name = "WorkshopIndonesia_test_4"
ee_region = ee.Geometry.Polygon(
        [[[113.54396581002183, -2.055453124813486],
          [113.54396581002183, -3.1145398246354588],
          [115.50227879830308, -3.1145398246354588],
          [115.50227879830308, -2.055453124813486]]]);

In [63]:
# Get explanatory composite closest to target date
image = get_explanatory_composite(
    target_date=target_date, 
    ee_region=ee_region, 
    max_days_offset=max_days_offset
).select(explain_vars)


In [64]:
len(explain_vars)

28

In [65]:
# Create a task and start it
task = ee.batch.Export.image.toDrive(
    image=image,
    description="test",
    folder="WorkshopIndonesia",
    fileNamePrefix=output_image_name,
    region=ee_region,
    scale=scale,
    maxPixels=1e13,
)

# Start the task
task.start()

# 2. Estimate GWL using the trained model

In [ ]:
from pathlib import Path
import rasterio as rio
import numpy as np
import dask.array as da
import joblib
from sklearn.ensemble import RandomForestClassifier

In [ ]:
explanatory_path = Path("data/11_explanatory_composites")
output_estimation_path = Path("data/12_estimated_gwl")

model_name = "data/10_models/Best_stations_Kalimantan.joblib"

# load the model with joblib
estimator = joblib.load(model_name)

In [34]:
import dask.array as da
import numpy as np
from multiprocessing import Pool


In [41]:
# Open the explanatory composite
with rio.open(explanatory_path/"WorkshopIndonesia_test_3.tif") as src:
    img_array = src.read()
    meta = src.meta

# Reshape to 2D array, shape will be (num_bands, height * width)
img_array = img_array.reshape((img_array.shape[0], -1)).T
print("Image shape", img_array.shape)

dask_array = da.from_array(img_array, chunks=(1000, 28))
chunks_list = [chunk.compute() for chunk in dask_array.to_delayed().ravel()]

Image shape (2747224, 28)


In [42]:
def predict_func(chunk):
    if chunk.shape[0] == 0 or np.isnan(chunk).any():
        return np.full(chunk.shape[0], np.nan)
    else:
        return estimator.predict(chunk)

def parallel_predict(chunk_list):
    with Pool() as pool:
        results = pool.map(predict_func, chunk_list)
    return np.concatenate(results)

# Assuming 'chunks' is a list of array chunks you want to predict
predicted_array = parallel_predict(chunks_list)
predicted_array = dask_array.map_blocks(predict_func, dtype=dask_array.dtype).compute(scheduler='processes')

/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/sklearn/ensemble/_base.py:214: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/sklearn/ensemble/_base.py:214: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-package

IndexError: tuple index out of range

In [52]:
# Open the explanatory composite
with rio.open(explanatory_path/"WorkshopIndonesia_test_3.tif") as src:
    img_array = src.read()
    meta = src.meta

# Reshape to 2D array, shape will be (num_bands, height * width)
# img_array = img_array.reshape((img_array.shape[0], -1)).T
# print("Image shape", img_array.shape)

In [53]:
# Transpose to have the feature bands last (shape: 1348, 2038, 28)
transposed_img = np.transpose(img_array, (1, 2, 0))

# Reshape it for prediction (shape: 2747224, 28)
reshaped_img = transposed_img.reshape(-1, 28)

print("Reshaped image shape:", reshaped_img.shape)

predicted_values = estimator.predict(reshaped_img)
# Reshape the predicted values back to original shape (shape: 1348, 2038)
predicted_image = predicted_values.reshape(1348, 2038)

print("Predicted image shape:", predicted_image.shape)



Reshaped image shape: (2747224, 28)


/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Predicted image shape: (1348, 2038)


In [56]:
# Reshape back to 2D grid
output_array = predicted_array.reshape((meta['height'], meta['width']))

# Update metadata for 1 band output
meta.update({'count': 1})

# Save to disk
with rio.open(output_estimation_path/f"{model_name}_{output_image_name}.tif", 'w', **meta) as dst:
    dst.write(output_array, 1)


RasterioIOError: Attempt to create new tiff file 'data/12_estimated_gwl/data/10_models/Best_stations_Kalimantan.joblib_WorkshopIndonesia_test_3.tif' failed: No such file or directory